In [127]:
"""
Скрипт для обработки данных  с портала трудвсем для БЦК
"""

'\nСкрипт для обработки данных  с портала трудвсем для БЦК\n'

In [20]:
import pandas as pd
import numpy as np
import openpyxl
import json
import ast

In [31]:
def extract_data_from_list_cell(cell:str,lst_need_keys:list):
    """
    Функция для извлечения данных из словаря в ячейке датафрейма
    """
    try:
        
        lst_lang = ast.literal_eval(cell) # превращаем в список
        if lst_lang:            
            out_str_lst = [] # создаем список содержащий выходные строки
            for lang_dict in lst_lang:
                lst_lang_str = [] # список для хранения значений извлеченных из словаря
                for idx,key in enumerate(lst_need_keys):                    
                     lst_lang_str.append(lang_dict.get(key,''))
                single_lang_str = ','.join(lst_lang_str) # создаем строку для одного языка
                out_str_lst.append(single_lang_str)
            return ';'.join(out_str_lst)
                               
        else:
            return 'Не указано'
        

    except FileNotFoundError:
        return 'Не удалось обработать содержимое ячейки'

    

In [3]:
region = 'Республика Бурятия'

In [12]:
# колонки которые нужно оставить и переименовать
dct_name_columns = {'busy_type':'Тип занятости','contact_person':'Контактное лицо','date_create':'Дата размещения вакансии',
                    'date_modify':'Дата изменения вакансии','education':'Требуемое образование','is_quoted':'Квотируемое место',
                   'other_vacancy_benefit':'Дополнительные бонусы','position_requirements':'Требования',
                    'position_responsibilities':'Обязанности','regionName':'Регион','required_experience':'Требуемый опыт работы в годах',
                    'retraining_capability':'Возможность переподготовки','required_certificates':'Требуемые доп. документы',
                   'required_drive_license':'Требуемые водительские права','retraining_grant':'Стипендия',
                    'retraining_grant_value':'Размер стипендии','salary':'Зарплата','salary_min':'Минимальная зарплата','salary_max':'Максимальная зарплата',
                    'schedule_type':'График работы','social_protected_ids':'Социально защищенная категория',
                   'source_type':'Источник вакансии','status':'Статус проверки вакансии','transport_compensation':'Компенсация транспорт',
                    'vacancy_address_additional_info':'Доп информация по адресу вакансии','vacancy_address':'Адрес вакансии',
                    'vacancy_address_latitude':'Долгота адрес вакансии','vacancy_address_longitude':'Широта адрес вакансии',
                   'vacancy_benefit_ids':'Бонусы','vacancy_name':'Вакансия','work_places':'Количество рабочих мест','professionalSphereName':'Сфера деятельности',
                    'full_company_name':'Полное название работодателя','company_inn':'ИНН работодателя','company':'Данные компании',
                    'languageKnowledge':'Данные по языкам','hardSkills':'Данные по хардскиллам','softSkills':'Данные по софтскиллам'}

In [5]:
df = pd.read_csv('data/vacancy.csv',encoding='UTF-8',sep='|',dtype=str)


In [6]:
company_df = pd.read_excel('data/company.xlsx',dtype=str)

In [7]:
company_df

,Краткое название организации,ИНН
0,УУАЗ,0323018510
1,Приборка,0323053578
2,Ника,7536165688
3,РХИ,7802772445


In [8]:
df.shape

(92697, 78)

In [136]:
#df = df[df['regionName'] == region]

In [9]:
df.shape

(92697, 78)

In [10]:
dct_df = dict() # словарь для хранения данных для каждого инн
used_name_sheet = set() # множество для хранения использованных названий листов

In [30]:
# Собираем датафреймы по ИНН
for idx,row in enumerate(company_df.itertuples()):
    name_company = row[1] # название компании
    inn_company = row[2] # инн компании
    temp_df = df[df['company_inn'] ==inn_company] # фильтруем по инн
    temp_df = temp_df[dct_name_columns.keys()]
    temp_df.rename(columns=dct_name_columns,inplace=True)
    # Начинаем извлекать данные из сложных колонок с json
    # данные по работодателю
    temp_df['КПП работодателя'] = temp_df['Данные компании'].apply(lambda x: json.loads(x).get('kpp','Не найден ключ'))
    temp_df['ОГРН работодателя'] = temp_df['Данные компании'].apply(lambda x: json.loads(x).get('ogrn','Не найден ключ'))
    temp_df['Контактный телефон работодателя'] = temp_df['Данные компании'].apply(lambda x: json.loads(x).get('phone','Не найден ключ'))
    temp_df['Email работодателя'] = temp_df['Данные компании'].apply(lambda x: json.loads(x).get('email','Не найден ключ'))
    temp_df['Профиль работодателя'] = temp_df['Данные компании'].apply(lambda x: json.loads(x).get('url','Не найден ключ'))
    
    # Обрабатываем колонку с языками
    temp_df['Знание языков'] = temp_df['Данные по языкам'].apply(lambda x:extract_data_from_list_cell(x,['code_language','level']))
    print(temp_df['Знание языков'])
    
    
    
    
    
    
    
    temp_df.to_excel(f'data/{name_company}.xlsx',index=False)
    

Китайский,;Английский,
Турецкий,Базовый;Английский,Свободный
Турецкий,Базовый;Английский,Свободный
Турецкий,Базовый;Английский,Свободный
Турецкий,Базовый;Английский,Свободный
Турецкий,Базовый


In [150]:
temp_df['Знание языков'].value_counts()

Series([], Name: Знание языков, dtype: int64)

In [142]:
print('Lindy Booth!')

Lindy Booth!
